# thefaceshop

In [1]:
from bs4 import BeautifulSoup as bs              # 데이터파싱 라이브러리
from selenium import webdriver
import json
import platform
import re
import time
import copy
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
def getNumber(string):
    numExtracter = re.compile('[0-9]+')
    return int(''.join(numExtracter.findall(string)))
    

In [3]:
def getCategoryList(url_products):
    driver.get(url_products)

    html = driver.page_source
    soup = bs(html,'html.parser')
    categories = soup.find('ul',{'class':'gnb_d2'}).find_all('li')

    categoryList = []
    for category in categories:
        categoryList.append(url_home + category.a['href'])
    return categoryList

In [4]:
def clickSeeMoreButton():
    while True:
        try:
            seeMoreButton = driver.find_element_by_link_text('상품 더보기') 
            driver.execute_script('arguments[0].scrollIntoView(true);', seeMoreButton)
            seeMoreButton.click()
            time.sleep(3)
        except:
            break

In [5]:
def getItemList():
    html = driver.page_source
    soup = bs(html,'html.parser')
    items = soup.find('ul',{'id':'prdtList'})
    itemList = []
    for i, item in enumerate(items):
        try:
            itemList.append(url_home + item.a['href'])
        except:
            pass
    return itemList

In [6]:
def getItem():
    html = driver.page_source
    soup = bs(html, 'html.parser')

    name = soup.find('div',{'class':'page_title_2'}).span.get_text()

    imageList = soup.find('ul',{'class':'pn_container slide_thumb_list'}).find_all('li')
    images = [image.img['src'] for image in imageList]

    categoryList = soup.find('ul',{'class':'loc'}).find_all('li')
    category = ''
    for c in categoryList:
        category += c.get_text() + ' > '        
    category = category[:-3]

    volume = soup.find('span',string='용량/사이즈')
    volume = volume.parent.parent.td.get_text().strip()

    salePrice = getNumber(soup.find('span',{'class':'prdt_price_2'}).get_text())

    originalPrice = soup.find('span',{'class':'prdt_price_3'})
    originalPrice = getNumber(originalPrice.get_text()) if originalPrice else salePrice

    brand = 'thefaceshop'
    url = driver.current_url


    item = {'name':'#', 'url':'#', 'image':'#', 'color':'#', 'category':'#', 
                   'salePrice':'#', 'originalPrice':'#', 'brand':'#','volume':'#'}
    item['name']=name
    item['image']=images
    item['category']=category
    item['volume']=volume
    item['originalPrice']=originalPrice
    item['salePrice']=salePrice
    item['brand']=brand
    item['url']=url

    colors = soup.find_all('ul',{'class':'euiSelectList'})
    colors = colors[1].find_all('li') if colors else ''
    colors = [color.get_text().strip() for color in colors]

    items=[]
    if colors:
        for color in colors:
            item_copy = copy.deepcopy(item)
            item_copy['color'] = color
            items.append(item_copy)
    else:
        items.append(item)
    display(items)
    return items

In [7]:
def writeJSON(jsonString, output_name='data.json'):
    with open(output_name,'w',encoding='UTF-8') as file:
        file.write(jsonString)


In [8]:
path = 'chromedriver.exe' if (platform.system() == 'Windows') else '/Users/jg/Desktop/develop/DataTeam/DataProcessing/product/crawling/chromedriver';
driver = webdriver.Chrome(path)

url_home = 'http://thefaceshop.com'
url_products = 'http://www.thefaceshop.com/mall/product/category.jsp?cate_seq=7'
driver.get(url_products)

In [9]:
start_time = time.time() 
#-------------------------------------------------------------#
categoryList = getCategoryList(url_products)
result = []
for category in categoryList:
    driver.get(category)
    clickSeeMoreButton()
    itemList = getItemList()
    totalItemNumber = getNumber(driver.find_element_by_xpath('//div[@class="result"]/span').text)
    print('상품 개수:', max(totalItemNumber,len(itemList))) # 총 상품 개수보다 표시된 상품 개수가 더 많은 경우도 존재
    if totalItemNumber > len(itemList):
        print('크롤링된 개수:',len(itemList))
        print('error')
        raise 
    cnt=0
    for item in itemList:
        driver.get(item)
        result += getItem()
        cnt+=1
        if cnt>3:
            break
#-------------------------------------------------------------#
print("--- %0.2f seconds ---" %(time.time() - start_time))

상품 개수: 141


[{'name': '더테라피 로얄메이드 오일 블렌딩 크림',
  'url': 'http://www.thefaceshop.com/mall/product/product-view.jsp?dpid=AF007871',
  'image': ['http://image.ethefaceshop.com/tfsshopWebSrc/upload/product/201711/AF007871_05_1.jpg',
   'http://image.ethefaceshop.com/tfsshopWebSrc/upload/product/201711/AF007871_05_2.jpg',
   'http://image.ethefaceshop.com/tfsshopWebSrc/upload/product/201711/AF007871_05_3.jpg'],
  'color': '#',
  'category': 'HOME > 유형별 > 스킨케어',
  'salePrice': 35000,
  'originalPrice': 35000,
  'brand': 'thefaceshop',
  'volume': '50ml'}]

[{'name': '닥터벨머 데일리리페어 모이스처라이저',
  'url': 'http://www.thefaceshop.com/mall/product/product-view.jsp?dpid=AF007892',
  'image': ['http://image.ethefaceshop.com/tfsshopWebSrc/upload/product/201711/AF007892_05_1.jpg'],
  'color': '#',
  'category': 'HOME > 유형별 > 스킨케어',
  'salePrice': 20000,
  'originalPrice': 20000,
  'brand': 'thefaceshop',
  'volume': '120ml'}]

[{'name': '닥터벨머 시카 펩타이트 앰플',
  'url': 'http://www.thefaceshop.com/mall/product/product-view.jsp?dpid=AF009075',
  'image': ['http://image.ethefaceshop.com/tfsshopWebSrc/upload/product/201809/AF009075_05_1.jpg'],
  'color': '#',
  'category': 'HOME > 유형별 > 스킨케어',
  'salePrice': 32000,
  'originalPrice': 32000,
  'brand': 'thefaceshop',
  'volume': '45ml'}]

[{'name': '닥터벨머 어드밴스드 시카 리커버리 크림',
  'url': 'http://www.thefaceshop.com/mall/product/product-view.jsp?dpid=AF007738',
  'image': ['http://image.ethefaceshop.com/tfsshopWebSrc/upload/product/201710/AF007738_05_1.jpg'],
  'color': '#',
  'category': 'HOME > 유형별 > 스킨케어',
  'salePrice': 32000,
  'originalPrice': 32000,
  'brand': 'thefaceshop',
  'volume': '50ml'}]

...

상품 개수: 3


[{'name': ' 제주 삼다수 2L  12개입',
  'url': 'http://www.thefaceshop.com/mall/product/product-view.jsp?dpid=AF008254',
  'image': ['http://image.ethefaceshop.com/tfsshopWebSrc/upload/product/201801/AF008254_05_1.jpg'],
  'color': '#',
  'category': 'HOME >  > 유형별',
  'salePrice': 15600,
  'originalPrice': 20600,
  'brand': 'thefaceshop',
  'volume': '2L*12'}]

[{'name': '제주 삼다수 500ml 40개입',
  'url': 'http://www.thefaceshop.com/mall/product/product-view.jsp?dpid=AF008256',
  'image': ['http://image.ethefaceshop.com/tfsshopWebSrc/upload/product/201801/AF008256_05_1.jpg'],
  'color': '#',
  'category': 'HOME >  > 유형별',
  'salePrice': 21700,
  'originalPrice': 36000,
  'brand': 'thefaceshop',
  'volume': '500ml*40'}]

[{'name': '제주 삼다수 500ml 20개입',
  'url': 'http://www.thefaceshop.com/mall/product/product-view.jsp?dpid=AF008255',
  'image': ['http://image.ethefaceshop.com/tfsshopWebSrc/upload/product/201801/AF008255_05_1.jpg'],
  'color': '#',
  'category': 'HOME >  > 유형별',
  'salePrice': 11100,
  'originalPrice': 18000,
  'brand': 'thefaceshop',
  'volume': '500ml*20'}]

--- 133.42 seconds ---


In [10]:
output = json.dumps(result,ensure_ascii=False, indent='\t')

writeJSON(output, output_name = 'thefaceshop.json')